In [ ]:
!pip install -q pyiceberg

In [ ]:
from pyiceberg.catalog.rest import RestCatalog
import logging

# logging.basicConfig(level=logging.DEBUG)

import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

CATALOG_URL = "http://server:8181/catalog"
DEMO_WAREHOUSE = "demo"

catalog = RestCatalog(
    name="my_catalog",
    warehouse=DEMO_WAREHOUSE,
    uri=CATALOG_URL,
    token="dummy",
)


In [ ]:
# Create a new namespace if it doesn't already exist
test_namespace = ("pyiceberg_namespace",)
if test_namespace not in catalog.list_namespaces():
    catalog.create_namespace(test_namespace)


In [ ]:
# Write data
test_table = ("pyiceberg_namespace", "my_table")
df = pd.DataFrame(
    {
        "id": [1, 2, 3],
        "data": ["a", "b", "c"],
    }
)
pa_df = pa.Table.from_pandas(df)

if test_table in catalog.list_tables(namespace=test_namespace):
    catalog.drop_table(test_table)

table = catalog.create_table(
    test_table,
    schema=pa_df.schema,
)

table.append(pa_df)

In [ ]:
# Read data
table = catalog.load_table(test_table)
table.scan().to_pandas()